This notebook presents my work for the **Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto** as part of **Applied Data Science Capstone**.

Import needed libraries.

In [72]:
import pandas as pd
import urllib.request
import html5lib
import numpy as np
from bs4 import BeautifulSoup

Get html content from the wikipedia page.

In [ ]:
p_can_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
fp = urllib.request.urlopen(p_can_url)
p_can_html = fp.read()
p_can_html = p_can_html.decode("utf8")
fp.close()
p_can = BeautifulSoup(p_can_html)

Get the postcode and neighbourhood table from the html content, and delete the rows where Borough equals Not assigned.

In [70]:
df_p_can = []
for tr in p_can.table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    df_p_can.append(row)
df_p_can=pd.DataFrame(df_p_can,columns=['Postcode','Borough','Neighbourhood'])
df_p_can=df_p_can[df_p_can['Borough']!='Not assigned']
df_p_can=df_p_can[df_p_can.Borough.notnull()]
df_p_can['Neighbourhood']=df_p_can['Neighbourhood'].replace({'\n':''}, regex=True)
df_p_can.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


If Neighbourhood is Not assigned, name it with the name of Borough.

In [74]:
df_p_can['Neighbourhood'] = np.where(df_p_can['Neighbourhood'] == 'Not assigned', df_p_can['Borough'], df_p_can['Neighbourhood'])

Combine Neighbourhoods with the same Borough into one cell, and split using commas.

In [76]:
df_p_can=df_p_can.groupby('Borough')['Neighbourhood'].apply(','.join).reset_index()

View table and the size of the table.

In [78]:
df_p_can.shape
print(df_p_can)

             Borough                                      Neighbourhood
0    Central Toronto  Lawrence Park,Roselawn,Davisville North,Forest...
1   Downtown Toronto  Harbourfront,Regent Park,Ryerson,Garden Distri...
2       East Toronto  The Beaches,The Danforth West,Riverdale,The Be...
3          East York  Woodbine Gardens,Parkview Hill,Woodbine Height...
4          Etobicoke  Islington Avenue,Cloverdale,Islington,Martin G...
5        Mississauga              Canada Post Gateway Processing Centre
6         North York  Parkwoods,Victoria Village,Lawrence Heights,La...
7       Queen's Park                                       Queen's Park
8        Scarborough  Rouge,Malvern,Highland Creek,Rouge Hill,Port U...
9       West Toronto  Dovercourt Village,Dufferin,Little Portugal,Tr...
10              York  Humewood-Cedarvale,Caledonia-Fairbanks,Del Ray...
